This notebook demonstrates use of the TorchRF DataLogger.  See LogDemoA for an overview and examples of writing out reference values.  In this notebook we load in those saved values and compare them to values created here. 

# Part 2: Compute test values and compare them with the reference values
We start with setting up the scene in TorchRF the same way we did for Sionna.

In [1]:
import numpy as np
import torchrf
from torchrf.rt.scene import load_scene
from torchrf.rt import PlanarArray, Transmitter, Camera

scene = load_scene(torchrf.rt.scene.munich)
resolution = [480, 320]
my_cam = Camera("my_cam", position=[-250, 250, 150], look_at=[-15, 30, 28])
scene.add(my_cam)
scene.tx_array = PlanarArray(num_rows=1, num_cols=1, vertical_spacing=0.5, horizontal_spacing=0.5,
                             pattern="tr38901",polarization="V")
tx = Transmitter(name="tx", position=[8.5, 21, 27])
scene.add(tx)

2024-01-12 21:51:21.636059: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Import the DataLogger. The default mode is 'break', which calls a breakpoint so that you can use the debugger to inspect the differences between the current code and the reference computation.  Since we're in a Jupyter Notebook right now, we will use mode 'print' instead which just prints out disagreements.

In [2]:
from torchrf.utils.datalogger import DataLogger
DataLogger().set_mode('print')

The simplest way to compare a variable to reference is just to call compare. You'll need to use the identical tag in order to compare to the correct variable.

In [3]:
logger = DataLogger()
logger.compare(tx, 'transmitter1')

At log point `transmitter1`:
  stored data object [data] has keys not in current data object [obj]:
   _color
  current data object [obj] has keys not in stored data object [data]:
   _trainable_position
   _trainable_orientationn
At log point `transmitter1:[_dtype]`:
  torch.complex64 does not match {'__doc__': '64-bit complex.'}
At log point `transmitter1:[_rdtype]`:
  torch.float32 does not match {'__doc__': '32-bit (single precision) floating-point.'}


Because the comparison was successful, we get a message that this tag "passed".  Note that this tx contains torch tensors while the reference tx contained tensorflow tensors, but both were converted to numpy ndarrays, so the comparison went through.  The comparison done on the Transmitter class was "deep", in that all fields (and fields of fields, etc) other than the Scene were compared.

Recall that in LogDemoA, the 753array contained a set of 3 vectors: {(3,3,3), (5,5,5), (7,7,7)} with the vectors stored in dim=1, but specfically they were stored with the 7's leftmost and the 3's rightmost.  Let's try a shuffle of those vectors:

In [4]:
a = np.array([[5, 3, 7],
              [5, 3, 7],
              [5, 3, 7]])
logger.compare(a, '753array')

At log point `753array`:
   tensors have different values


As expected, that comparison fails because the tensors are not equivalent.  But if we know that we only want to know 
whether these are the same set of unique vectors, we can use the `unique` keyword.  In this case we will also need to 
specify the dim (it's the same one used as an argument to the tf or torch `unique` functions).

In [5]:
logger.compare(a, '753array', unique=True, dim=1)

At log point `753array`: passed


This only works if the vectors really were unique, so the following fails, for example:

In [6]:
logger.compare(np.array([[5, 7, 3, 7],
                         [5, 7, 3, 7],
                         [5, 7, 3, 7]]), '753array', unique=True, dim=1)

At log point `753array`:
   tensors have different shapes


We build up tag stacks for reading and comparing in the same way as when writing.  The data will only be compared when the entire stack matches.

In [7]:
def foo(x):
    y = x * x
    logger.compare(y, 'intermediate foo')
    z = y / 2
    return z

with DataLogger().push('foo A') as logger:
    z1 = foo(7)
    logger.compare(z1, 'first returned z')
    with logger.push('foo 2') as logger:
        z2 = foo(5)
        with logger.push('foo 3') as logger:
            z3 = foo(3)

At log point `foo A:intermediate foo`: passed
At log point `foo A:first returned z`: passed
At log point `foo A:foo 2:intermediate foo`: passed
At log point `foo A:foo 2:foo 3:intermediate foo`: passed


Recall that LogDemoA wrote a random matrix.  We will obviously fail on that:

In [8]:
with DataLogger().push('rand') as logger:
    a = np.random.rand(3, 5)
    logger.compare(a, 'A')

At log point `rand:A`:
   tensors have different values


But we might want to make sure that our function `foo` operates on that random matrix the same way that the original `foo`.  We can do that by explicitly loading in the saved data:

In [9]:
with DataLogger().push('rand') as logger:
    a = logger.get('A')
    z = foo(a)
    logger.compare(z, 'foo A')

At log point `rand:intermediate foo`: passed
At log point `rand:foo A`: passed


# Part 3: Other Details

It was mentioned above that `logs` and `logs/index.json` are the default storage places.  These can be overridden using `DataLogger().set_dir(log_directory)` and `DataLogger().set_index(index_file_spec)`.  These can be absolute or relative file paths.

`pickle` only pickles certain data types.  `DataLogger.prepickle()` converts data objects into pickleable objects.  It covers the cases that have come up so far, but might not cover everything we want to pickle and so may need to be extended.

As described above, the DataLogger tool requires us to add code to the actual source that we want to test, so it isn't really used like unit test code unless we only want to compare input and output values without intermediate values.  Because the Sionna methods contain long internal computations, we will want the intermediate values, requiring us to put DataLogger calls into the Sionna source code as well as the TorchRF code.  Some of the work to be done is to try to guess how the different variables align, and this gives us a way to check our guesses.  After we have finished debugging, we will presumably want to go through and eliminate DataLogger calls before making code public.